In [1]:
import pandas as pd
import numpy as np

## Data 불러오기

In [2]:
hanjeon = pd.read_excel("급속충전이용내역_한전_18.01-19.10.xlsx")
hj = hanjeon.copy()
hj.head(3)

,번호,용도별,충전소,주소,충전소 구축일,충전구분,충전량(kWh),시작일시
0,1,공용,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,급속,17.21,2018-01-01 09:19:36
1,2,공용,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,급속,12.86,2018-01-02 09:16:04
2,3,공용,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,급속,16.10,2018-01-04 14:00:49


In [3]:
hwangyeong = pd.read_excel("급속충전이용내역_환경부_18.01-19.10.xlsx")
hg = hwangyeong.copy()
hg.head(3)

,번호,충전소대분류,충전소명,주소,설치년도,충전기타입,충전량(kWh),충전시작시간,충전종료시간
0,1,교육문화시설,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,2017,DC차데모+AC3상+DC콤보,7.02,20180101032052,20180101034847
1,2,공공시설,중부지방고용노동청 별관 앞,인천광역시 남동구 구월3동 1111-6 중부지방고용노동청 별관 앞,2017,DC차데모+AC3상+DC콤보,2.05,20180101041556,20180101042036
2,3,주차시설,강화풍물시장 공영주차장,인천광역시 강화군 강화읍 갑곳리 849 고엽제전우회 건물 옆,2017,DC차데모+AC3상+DC콤보,16.95,20180101043214,20180101050354


## Data의 Columns명 일치

In [4]:
hj.rename( columns = {
    '충전소 구축일' : '설치년도'
}, inplace = True)

hg.rename( columns = {
    '충전소명' : '충전소',
    '충전시작시간' : '시작일시',
    '충전종료시간' : '종료일시'
}, inplace = True)

In [5]:
hj.head(1)

,번호,용도별,충전소,주소,설치년도,충전구분,충전량(kWh),시작일시
0,1,공용,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,급속,17.21,2018-01-01 09:19:36


In [6]:
hg.head(1)

,번호,충전소대분류,충전소,주소,설치년도,충전기타입,충전량(kWh),시작일시,종료일시
0,1,교육문화시설,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,2017,DC차데모+AC3상+DC콤보,7.02,20180101032052,20180101034847


## 필요 데이터 추출 및 재구성

### 충전소 아파트용은 제외

In [7]:
hj2 = hj[hj['용도별'] == '공용'] # hj['용도별'] != '아파트용' 
hj2['용도별'].unique()

array(['공용'], dtype=object)

### Data 재구성

In [8]:
hj2 = hj2.loc[: , ['충전소', '주소', '설치년도', '충전량(kWh)', '시작일시']]
hg2 = hg.loc[: , ['충전소', '주소', '설치년도', '충전량(kWh)', '시작일시']]

In [9]:
df = pd.concat([hj2, hg2], axis = 0)
print(len(df), len(df.columns))
df.head(1)

83009 5


,충전소,주소,설치년도,충전량(kWh),시작일시
0,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36


In [10]:
df['충&주'] = list(map(lambda x, y : x + ' / ' + y, df['충전소'], df['주소']))
print(len(df['충전소'].unique()) , len(df['주소'].unique()), len(df['충&주'].unique()))

84 85 85


In [11]:
df.drop_duplicates(['주소'], keep = 'first').groupby(['충전소']).count()[df.drop_duplicates(['주소'], keep = 'first').groupby(['충전소']).count()['충&주'] >= 2]

,주소,설치년도,충전량(kWh),시작일시,충&주
충전소,,,,,
롯데마트 영종도점,2,2,2,2,2


### => 충전소는 84개인데 주소가 85개인 이유

In [12]:
df[df['충전소'] == '롯데마트 영종도점']['주소'].unique()

array(['인천광역시 중구 흰바위로 51 , 지상주차장 4층 455구역', '인천광역시 중구 흰바위로 51'],
      dtype=object)

In [13]:
df.loc[df['충전소'] == '롯데마트 영종도점', '주소'] = ['인천광역시 중구 흰바위로 51 지상주차장 4층'] * len(df.loc[df['충전소'] == '롯데마트 영종도점', '주소'])


In [14]:
df[df['충전소'] == '롯데마트 영종도점']['주소'].unique()

array(['인천광역시 중구 흰바위로 51 지상주차장 4층'], dtype=object)

In [15]:
df['충&주'] = list(map(lambda x, y : x + ' / ' + y, df['충전소'], df['주소']))
print(len(df['충전소'].unique()) , len(df['주소'].unique()), len(df['충&주'].unique()))

84 84 84


### => 완료

In [16]:
df.index = range(len(df))

In [17]:
df.head(1)

,충전소,주소,설치년도,충전량(kWh),시작일시,충&주
0,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)"


## 충전소 고유정보로 이루어진 DataFrame에 총 충전량, 빈도 Column을 추가

### 충전소 위치data

In [18]:
ch_geo = pd.read_csv("충전소명_주소_좌표변환.csv", encoding = "CP949")
ch_geo.head(1)

,충전소,주소,새우편번호,지번주소 상위부분,지번주소 하위부분,새주소 상위부분,새주소 하위부분,주소정제결과코드,지번코드,새주소 건물관리번호,...,Y좌표,새주소 길이름,새주소 주건물번호,새주소 부건물번호,구우편번호,도로명길코드,지상지하구분,새주소건물명,블록코드,새주소변환코드
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",22725.0,인천광역시 서구 심곡동,239-9번지 후문주차타워 서구청 지상1층,인천광역시 서구 탁옥로51번길,"26, 후문주차타워 지상1층 서구청(심곡동)",정좌표,2.826010e+18,2.826010e+24,...,37.546065,탁옥로51번길,26,0,404833.0,2.826040e+11,0,서구청제1주차장,NaN,지번확인된 새주소


In [19]:
ch_geo.columns

Index(['충전소', '주소', '새우편번호', '지번주소 상위부분', '지번주소 하위부분', '새주소 상위부분', '새주소 하위부분',
       '주소정제결과코드', '지번코드', '새주소 건물관리번호', '광역시도코드', '광역시도명', '시군구코드', '시군구명',
       '비자치행정구역코드', '3차행정구역명', '법정동코드', '법정동명', '행정동코드', '행정동명', '리명', '건물명',
       '동명', '호명', '층', '지목', '주번지', '부번지', 'X좌표', 'Y좌표', '새주소 길이름',
       '새주소 주건물번호', '새주소 부건물번호', '구우편번호', '도로명길코드', '지상지하구분', '새주소건물명', '블록코드',
       '새주소변환코드'],
      dtype='object')

In [20]:
ch_geo = ch_geo.loc[:, ['충전소', '주소', 'X좌표','Y좌표', '시군구명']]

In [21]:
len(ch_geo)

170

In [22]:
len(df)

83009

In [23]:
len(df['충전소'].unique())

84

In [24]:
(set(ch_geo['충전소']) & set(df['충전소'].unique()))

{'G-Tower(경제자유구역청)',
 'LH인천본부',
 '㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구',
 '가좌2동 행정복지센터',
 '갈산역 공영주차장',
 '강화군청',
 '강화지사',
 '강화풍물시장 공영주차장',
 '검단2공영주차장',
 '계산1택지 공영주차장',
 '계양구 공영주차장',
 '계양구청',
 '남동복지관',
 '남인천지사',
 '늘솔길공원 3주차장',
 '덕산국민여가캠핑장',
 '도원역 부근 공영주차장',
 '동구청앞 공영주차장',
 '롯데마트 검단점',
 '롯데마트 부평역점',
 '롯데마트 연수점',
 '롯데마트 영종도점',
 '롯데마트 청라점',
 '마니산국민관광지',
 '모탱이공원',
 '부원 공영주차장',
 '부평국민체육센터',
 '부평깡시장 공영주차장',
 '부평아트센터',
 '삼산농산물도매시장',
 '서곶근린공원 제1주차장',
 '서림초교 뒤 공영주차장',
 '서운체육공원 주차장',
 '소래제1주차장',
 '송도골프',
 '송도컨벤시아',
 '송림체육관',
 '신기남부종합시장 주차장',
 '신문리 공영주차장',
 '십리포해수욕장 주차장',
 '아트센터인천',
 '연세대학교 국제캠퍼스',
 '연수구청',
 '열우물경기장',
 '영종대교 휴게소 주차장',
 '영흥수산물직판장 주차장',
 '온수리 공영 주차장',
 '온수리 제1공영주차장',
 '용유동사무소',
 '월미공원 제2주차장',
 '이마트 동인천점',
 '이마트 연수점',
 '인천경제산업정보테크노파크',
 '인천공항 정부합동청사',
 '인천공항물류단지 행정지원센터',
 '인천광역시 교통연수원',
 '인천교통공사 본사',
 '인천나비공원 주차장',
 '인천서구청',
 '인천서구청 제1주차장',
 '인천서구청소년수련관',
 '인천시청',
 '인천아시아드주경기장 D1주차장',
 '인천아시안게임 남동경기장 1주차장',
 '인천업사이클 에코센터',
 '인천전력관리처',
 '인천정보산업테크노파크',
 '인천직할',
 '인천항 갑문홍보관',
 '인천항 크루즈터미널',
 '장수서창

## 아파트용아닌 충전소만 추출

In [25]:
ch_geo2 = ch_geo[ ch_geo['충전소'].isin(list(df['충전소'].unique())) ]
ch_geo2.head(1)

,충전소,주소,X좌표,Y좌표,시군구명
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",126.675984,37.546065,서구


### df에 좌표와 시군구명 추가

In [26]:
print(len(df), len(df.columns), len(df['충전소'].unique()))
df.head(1)

83009 6 84


,충전소,주소,설치년도,충전량(kWh),시작일시,충&주
0,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)"


In [27]:
print(len(ch_geo2), len(ch_geo2.columns))
ch_geo2.head(1)

85 5


,충전소,주소,X좌표,Y좌표,시군구명
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",126.675984,37.546065,서구


In [28]:
ch_geo2.loc[ch_geo2['충전소'] == '롯데마트 영종도점', '주소'] = '인천광역시 중구 흰바위로 51 지상주차장 4층'

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [29]:
ch_geo2[ch_geo2['충전소'] == '롯데마트 영종도점']
ch_geo2 = ch_geo2.drop_duplicates(['주소'], keep = 'first')
ch_geo2.index = range(len(ch_geo2))
ch_geo2.head(1)

,충전소,주소,X좌표,Y좌표,시군구명
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",126.675984,37.546065,서구


### 충전이력 빈도

In [30]:
df.groupby(['충전소']).count()

,주소,설치년도,충전량(kWh),시작일시,충&주
충전소,,,,,
G-Tower(경제자유구역청),1261,1261,1261,1261,1261
LH인천본부,3078,3078,3078,3078,3078
㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,651,651,651,651,651
가좌2동 행정복지센터,400,400,400,400,400
갈산역 공영주차장,187,187,187,187,187
강화군청,1397,1397,1397,1397,1397
강화지사,504,504,504,504,504
강화풍물시장 공영주차장,1754,1754,1754,1754,1754
검단2공영주차장,2510,2510,2510,2510,2510


### 충전소별 충전 일수

In [31]:
df.head(1)

,충전소,주소,설치년도,충전량(kWh),시작일시,충&주
0,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)"


In [32]:
df.loc[2, '시작일시']

'2018-01-04 14:00:49'

In [33]:
df2 = df.copy()
df2['충전일'] = list(map(lambda x : str(x) , df2['시작일시']))
df2['충전일'] = list(map(lambda x : x.replace('-', ''), df2['충전일']))
df2['충전일'] = list(map(lambda x : x[2:8] , df2['충전일']))
df2['충전일'] = list(map(lambda x : int(x) , df2['충전일']))
df2.head(1)

,충전소,주소,설치년도,충전량(kWh),시작일시,충&주,충전일
0,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",180101


In [34]:
temp = df2.sort_values(['충전소', '충전일'], ascending = True).loc[:,['충전소','충전일']]
temp.head(3)

,충전소,충전일
19911,G-Tower(경제자유구역청),180423
19912,G-Tower(경제자유구역청),180423
19913,G-Tower(경제자유구역청),180424


In [35]:
temp['충전소'].unique()

array(['G-Tower(경제자유구역청)', 'LH인천본부', '㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구',
       '가좌2동 행정복지센터', '갈산역 공영주차장', '강화군청', '강화지사', '강화풍물시장 공영주차장',
       '검단2공영주차장', '계산1택지 공영주차장', '계양구 공영주차장', '계양구청', '남동복지관', '남인천지사',
       '늘솔길공원 3주차장', '덕산국민여가캠핑장', '도원역 부근 공영주차장', '동구청앞 공영주차장',
       '롯데마트 검단점', '롯데마트 부평역점', '롯데마트 연수점', '롯데마트 영종도점', '롯데마트 청라점',
       '마니산국민관광지', '모탱이공원', '부원 공영주차장', '부평국민체육센터', '부평깡시장 공영주차장',
       '부평아트센터', '삼산농산물도매시장', '서곶근린공원 제1주차장', '서림초교 뒤 공영주차장',
       '서운체육공원 주차장', '소래제1주차장', '송도골프', '송도컨벤시아', '송림체육관', '신기남부종합시장 주차장',
       '신문리 공영주차장', '십리포해수욕장 주차장', '아트센터인천', '연세대학교 국제캠퍼스', '연수구청',
       '열우물경기장', '영종대교 휴게소 주차장', '영흥수산물직판장 주차장', '온수리 공영 주차장',
       '온수리 제1공영주차장', '용유동사무소', '월미공원 제2주차장', '이마트 동인천점', '이마트 연수점',
       '인천경제산업정보테크노파크', '인천공항 정부합동청사', '인천공항물류단지 행정지원센터', '인천광역시 교통연수원',
       '인천교통공사 본사', '인천나비공원 주차장', '인천서구청', '인천서구청 제1주차장', '인천서구청소년수련관',
       '인천시청', '인천아시아드주경기장 D1주차장', '인천아시안게임 남동경기장 1주차장', '인천업사이클 에코센터',
       '인천전력관리처', '인천정보산업테크노파크', '인천직할'

In [36]:
temp.groupby(['충전소','충전일']).count().index.levels[0][0]

'G-Tower(경제자유구역청)'

In [37]:
temp.groupby(['충전소', '충전일']).count().index.levels

FrozenList([['G-Tower(경제자유구역청)', 'LH인천본부', '㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구', '가좌2동 행정복지센터', '갈산역 공영주차장', '강화군청', '강화지사', '강화풍물시장 공영주차장', '검단2공영주차장', '계산1택지 공영주차장', '계양구 공영주차장', '계양구청', '남동복지관', '남인천지사', '늘솔길공원 3주차장', '덕산국민여가캠핑장', '도원역 부근 공영주차장', '동구청앞 공영주차장', '롯데마트 검단점', '롯데마트 부평역점', '롯데마트 연수점', '롯데마트 영종도점', '롯데마트 청라점', '마니산국민관광지', '모탱이공원', '부원 공영주차장', '부평국민체육센터', '부평깡시장 공영주차장', '부평아트센터', '삼산농산물도매시장', '서곶근린공원 제1주차장', '서림초교 뒤 공영주차장', '서운체육공원 주차장', '소래제1주차장', '송도골프', '송도컨벤시아', '송림체육관', '신기남부종합시장 주차장', '신문리 공영주차장', '십리포해수욕장 주차장', '아트센터인천', '연세대학교 국제캠퍼스', '연수구청', '열우물경기장', '영종대교 휴게소 주차장', '영흥수산물직판장 주차장', '온수리 공영 주차장', '온수리 제1공영주차장', '용유동사무소', '월미공원 제2주차장', '이마트 동인천점', '이마트 연수점', '인천경제산업정보테크노파크', '인천공항 정부합동청사', '인천공항물류단지 행정지원센터', '인천광역시 교통연수원', '인천교통공사 본사', '인천나비공원 주차장', '인천서구청', '인천서구청 제1주차장', '인천서구청소년수련관', '인천시청', '인천아시아드주경기장 D1주차장', '인천아시안게임 남동경기장 1주차장', '인천업사이클 에코센터', '인천전력관리처', '인천정보산업테크노파크', '인천직할', '인천항 갑문홍보관', '인천항 크루즈터미널', '장수서창 공영주차장', '중부지방고용노동청 별관 앞', '차이나타운 주차장', '청라에너지', '청라중앙호수공원

In [38]:
s = list(temp['충전소'].unique())
ch_day_cont = []

for i in s :
    ch_day_cont.append(len(temp[temp['충전소'] == i]['충전일'].unique()))
    
len(ch_day_cont)

84

### 총충전량

In [39]:
df.groupby(['충전소']).sum()

,충전량(kWh)
충전소,
G-Tower(경제자유구역청),30259.44
LH인천본부,63815.83
㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65
가좌2동 행정복지센터,7387.11
갈산역 공영주차장,3879.41
강화군청,29291.07
강화지사,10877.10
강화풍물시장 공영주차장,27766.94
검단2공영주차장,44358.60


In [40]:
ch_tot = df.groupby(['충전소']).sum()
ch_tot['충전소'] = ch_tot.index
ch_tot.index = range(len(ch_tot))
ch_tot = ch_tot.iloc[:, [1,0]]
ch_tot.head(10)

,충전소,충전량(kWh)
0,G-Tower(경제자유구역청),30259.44
1,LH인천본부,63815.83
2,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65
3,가좌2동 행정복지센터,7387.11
4,갈산역 공영주차장,3879.41
5,강화군청,29291.07
6,강화지사,10877.10
7,강화풍물시장 공영주차장,27766.94
8,검단2공영주차장,44358.60
9,계산1택지 공영주차장,27959.93


In [41]:
ch_tot['충전일수'] = ch_day_cont
ch_tot.head(5)

,충전소,충전량(kWh),충전일수
0,G-Tower(경제자유구역청),30259.44,408
1,LH인천본부,63815.83,612
2,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65,375
3,가좌2동 행정복지센터,7387.11,196
4,갈산역 공영주차장,3879.41,131


## 충전소 위치데이터와 충전 일수와 총 충전량 결합

In [42]:
ch_tot

,충전소,충전량(kWh),충전일수
0,G-Tower(경제자유구역청),30259.44,408
1,LH인천본부,63815.83,612
2,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65,375
3,가좌2동 행정복지센터,7387.11,196
4,갈산역 공영주차장,3879.41,131
5,강화군청,29291.07,517
6,강화지사,10877.10,342
7,강화풍물시장 공영주차장,27766.94,597
8,검단2공영주차장,44358.60,630
9,계산1택지 공영주차장,27959.93,606


In [43]:
ch_geo2

,충전소,주소,X좌표,Y좌표,시군구명
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",126.675984,37.546065,서구
1,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,126.667272,37.381511,연수구
2,중부지방고용노동청 별관 앞,인천광역시 남동구 구월3동 1111-6 중부지방고용노동청 별관 앞,126.699538,37.451133,남동구
3,강화풍물시장 공영주차장,인천광역시 강화군 강화읍 갑곳리 849 고엽제전우회 건물 옆,126.492866,37.741422,강화군
4,영종대교 휴게소 주차장,인천광역시 서구 경서동 산 257-4 지상 버스주차장 옆,126.607081,37.553904,서구
5,인천공항 정부합동청사,"인천광역시 중구 공항로424번길 47 , 정문 주차장",126.461336,37.440390,중구
6,검단2공영주차장,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,126.675023,37.592024,서구
7,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",126.499131,37.741871,강화군
8,계산1택지 공영주차장,인천광역시 계양구 계산동 1060 구두수선방 옆,126.736758,37.540990,계양구
9,계양구 공영주차장,"인천광역시 계양구 봉오대로 679-1 , 작전역 2번 출구 100m 앞",126.724460,37.530911,계양구


In [44]:
df2 = pd.merge( ch_tot, ch_geo2 , how = 'left', on = ['충전소'])
print( len(df2), len(df2.columns) )
df2.head(1)

84 7


,충전소,충전량(kWh),충전일수,주소,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,408,인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구


In [45]:
df2.to_csv("2019.12.02 전기차충전소 좌표 및 정보.csv", encoding = "CP949", index = False)

## 버퍼 km별 전기차수

In [46]:
km_ele = pd.read_csv("충전소명_주소_좌표변환_버퍼km별_전기차수.csv",encoding = "CP949")
km_ele.head(3)

,충전소,주소,시군구명,X좌표,Y좌표,CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,44,75,136,197,273
1,인천 학익풍림아이원 아파트,인천광역시 미추홀구 매소홀로 340,미추홀구,126.66147,37.44042,9,61,163,250,393
2,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,연수구,126.66727,37.38151,29,121,208,320,371


## df2와 km_ele join

In [47]:
df3 = pd.merge(df2, km_ele, on = '충전소')
df3.head(3)

,충전소,충전량(kWh),충전일수,주소_x,X좌표_x,Y좌표_x,시군구명_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km
0,G-Tower(경제자유구역청),30259.44,408,인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구,인천광역시 연수구 아트센터대로 175,연수구,126.63379,37.39574,56,127,213,288,358
1,LH인천본부,63815.83,612,인천광역시 남동구 논현로46번길 23,126.712389,37.400860,남동구,인천광역시 남동구 논현로46번길 23,남동구,126.71239,37.40086,16,83,110,174,275
2,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65,375,인천광역시 서구 원삭로 160 주차장 입구 우측 완속충전기 옆,126.666335,37.563021,서구,인천광역시 서구 원삭로 160 주차장 입구 우측 완속충전기 옆,서구,126.66633,37.56302,20,55,111,187,257


## 총충전량과 km별 버퍼 내에 전기차수 상관관계

In [48]:
df3_pre = df3.iloc[:, [0,2,1, 11,12,13,14,15]]
df3_pre

,충전소,충전일수,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km
0,G-Tower(경제자유구역청),408,30259.44,56,127,213,288,358
1,LH인천본부,612,63815.83,16,83,110,174,275
2,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,375,10538.65,20,55,111,187,257
3,가좌2동 행정복지센터,196,7387.11,6,85,134,294,481
4,갈산역 공영주차장,131,3879.41,92,154,255,309,345
5,강화군청,517,29291.07,10,17,34,36,44
6,강화지사,342,10877.10,8,18,31,37,41
7,강화풍물시장 공영주차장,597,27766.94,9,21,33,38,43
8,검단2공영주차장,630,44358.60,25,36,71,112,134
9,계산1택지 공영주차장,606,27959.93,89,114,157,218,291


In [49]:
df3_pre.iloc[:,2:].corr()

,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km
충전량(kWh),1.000000,0.245825,0.286968,0.199633,0.164595,0.142093
CarNum_1km,0.245825,1.000000,0.640487,0.536532,0.469607,0.383542
CarNum_2km,0.286968,0.640487,1.000000,0.912466,0.773877,0.634143
CarNum_3km,0.199633,0.536532,0.912466,1.000000,0.905746,0.763568
CarNum_4km,0.164595,0.469607,0.773877,0.905746,1.000000,0.930457
CarNum_5km,0.142093,0.383542,0.634143,0.763568,0.930457,1.000000


## 총충전량/충전일수 와 거리별 버퍼내 전기차수 상관관계

In [50]:
df3_pre['일수당충전량'] = df3_pre['충전량(kWh)'] / df3_pre['충전일수']
df3_pre.head(3)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,충전소,충전일수,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량
0,G-Tower(경제자유구역청),408,30259.44,56,127,213,288,358,74.165294
1,LH인천본부,612,63815.83,16,83,110,174,275,104.274232
2,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,375,10538.65,20,55,111,187,257,28.103067


In [51]:
df3_pre['일수당충전량'].isnull().sum()

0

In [52]:
df3_pre.iloc[:,3:].corr()

,CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량
CarNum_1km,1.000000,0.640487,0.536532,0.469607,0.383542,0.264436
CarNum_2km,0.640487,1.000000,0.912466,0.773877,0.634143,0.294193
CarNum_3km,0.536532,0.912466,1.000000,0.905746,0.763568,0.222485
CarNum_4km,0.469607,0.773877,0.905746,1.000000,0.930457,0.202231
CarNum_5km,0.383542,0.634143,0.763568,0.930457,1.000000,0.186152
일수당충전량,0.264436,0.294193,0.222485,0.202231,0.186152,1.000000


In [53]:
pd.set_option("display.max_rows", 100)
df3_pre['충전소']

0               G-Tower(경제자유구역청)
1                         LH인천본부
2     ㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구
3                    가좌2동 행정복지센터
4                      갈산역 공영주차장
5                           강화군청
6                           강화지사
7                   강화풍물시장 공영주차장
8                       검단2공영주차장
9                    계산1택지 공영주차장
10                     계양구 공영주차장
11                          계양구청
12                         남동복지관
13                         남인천지사
14                    늘솔길공원 3주차장
15                     덕산국민여가캠핑장
16                  도원역 부근 공영주차장
17                    동구청앞 공영주차장
18                      롯데마트 검단점
19                     롯데마트 부평역점
20                      롯데마트 연수점
21                     롯데마트 영종도점
22                     롯데마트 영종도점
23                      롯데마트 청라점
24                      마니산국민관광지
25                         모탱이공원
26                      부원 공영주차장
27                      부평국민체육센터
28                   부평깡시장 공영주차장
29                        부평아트센터
30        

In [54]:
df3_pre[ df3_pre['충전소'].isin(['마트', '홈플러스'])]

,충전소,충전일수,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량


In [55]:
mart_li = []

In [56]:
df3_pre = df3_pre.drop_duplicates(['충전소'], keep = 'first').sort_values(['충전소'])
df3_pre.index = range(len(df3_pre))
df3_pre

,충전소,충전일수,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량
0,G-Tower(경제자유구역청),408,30259.44,56,127,213,288,358,74.165294
1,LH인천본부,612,63815.83,16,83,110,174,275,104.274232
2,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,375,10538.65,20,55,111,187,257,28.103067
3,가좌2동 행정복지센터,196,7387.11,6,85,134,294,481,37.689337
4,갈산역 공영주차장,131,3879.41,92,154,255,309,345,29.613817
5,강화군청,517,29291.07,10,17,34,36,44,56.655841
6,강화지사,342,10877.10,8,18,31,37,41,31.804386
7,강화풍물시장 공영주차장,597,27766.94,9,21,33,38,43,46.510787
8,검단2공영주차장,630,44358.60,25,36,71,112,134,70.410476
9,계산1택지 공영주차장,606,27959.93,89,114,157,218,291,46.138498


In [57]:
mart_li = []

for i in list(df3_pre['충전소']) :
    if( (i.find('마트') != -1 ) | (i.find('홈플러스') != -1)) : 
        mart_li.append(i)
mart_li

['롯데마트 검단점',
 '롯데마트 부평역점',
 '롯데마트 연수점',
 '롯데마트 영종도점',
 '롯데마트 청라점',
 '이마트 동인천점',
 '이마트 연수점',
 '하나로마트 인천점',
 '홈플러스 가좌점',
 '홈플러스 계산점',
 '홈플러스 구월점',
 '홈플러스 인천연수점',
 '홈플러스 인하점',
 '홈플러스 작전점']

In [58]:
df3_mart = df3_pre[df3_pre['충전소'].isin(mart_li)]

In [59]:
df3_Nomart = df3_pre[df3_pre['충전소'].isin(list(set(df3_pre['충전소']) - set(mart_li)))]

In [60]:
df3_mart

,충전소,충전일수,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량
18,롯데마트 검단점,499,28768.72,16,46,72,113,137,57.652745
19,롯데마트 부평역점,501,20124.35,19,82,175,256,369,40.168363
20,롯데마트 연수점,335,16488.97,46,91,135,278,459,49.220806
21,롯데마트 영종도점,516,29106.06,16,35,41,61,71,56.407093
22,롯데마트 청라점,524,51481.52,25,80,101,175,228,98.247176
50,이마트 동인천점,235,17285.21,9,98,135,206,279,73.554085
51,이마트 연수점,356,7994.79,37,72,164,301,465,22.457275
75,하나로마트 인천점,208,6853.27,13,153,254,297,326,32.948413
78,홈플러스 가좌점,126,1813.40,44,97,130,214,439,14.392063
79,홈플러스 계산점,318,11688.85,90,113,158,254,298,36.757390


In [61]:
df3_Nomart

,충전소,충전일수,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량
0,G-Tower(경제자유구역청),408,30259.44,56,127,213,288,358,74.165294
1,LH인천본부,612,63815.83,16,83,110,174,275,104.274232
2,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,375,10538.65,20,55,111,187,257,28.103067
3,가좌2동 행정복지센터,196,7387.11,6,85,134,294,481,37.689337
4,갈산역 공영주차장,131,3879.41,92,154,255,309,345,29.613817
5,강화군청,517,29291.07,10,17,34,36,44,56.655841
6,강화지사,342,10877.10,8,18,31,37,41,31.804386
7,강화풍물시장 공영주차장,597,27766.94,9,21,33,38,43,46.510787
8,검단2공영주차장,630,44358.60,25,36,71,112,134,70.410476
9,계산1택지 공영주차장,606,27959.93,89,114,157,218,291,46.138498


In [62]:
print(len(df3_pre), len(df3_mart), len(df3_Nomart), len(df3_mart) + len(df3_Nomart))

84 14 70 84


## 대형마트와 아닌 곳 km별 버퍼 내에 전기차수 상관관계

In [63]:
df3_mart.iloc[:, 2:].corr()

,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량
충전량(kWh),1.000000,-0.179858,-0.423055,-0.423095,-0.368772,-0.389413,0.928805
CarNum_1km,-0.179858,1.000000,0.118553,0.008193,0.272618,0.290472,-0.197264
CarNum_2km,-0.423055,0.118553,1.000000,0.893846,0.702990,0.397442,-0.235815
CarNum_3km,-0.423095,0.008193,0.893846,1.000000,0.851141,0.577206,-0.275357
CarNum_4km,-0.368772,0.272618,0.702990,0.851141,1.000000,0.848678,-0.213758
CarNum_5km,-0.389413,0.290472,0.397442,0.577206,0.848678,1.000000,-0.277426
일수당충전량,0.928805,-0.197264,-0.235815,-0.275357,-0.213758,-0.277426,1.000000


In [64]:
df3_Nomart.iloc[:, 2:].corr()

,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량
충전량(kWh),1.000000,0.322388,0.380151,0.291567,0.237531,0.229957,0.843069
CarNum_1km,0.322388,1.000000,0.703588,0.605557,0.486301,0.378959,0.367808
CarNum_2km,0.380151,0.703588,1.000000,0.913717,0.772145,0.642861,0.384234
CarNum_3km,0.291567,0.605557,0.913717,1.000000,0.908266,0.777299,0.313454
CarNum_4km,0.237531,0.486301,0.772145,0.908266,1.000000,0.938706,0.273638
CarNum_5km,0.229957,0.378959,0.642861,0.777299,0.938706,1.000000,0.280250
일수당충전량,0.843069,0.367808,0.384234,0.313454,0.273638,0.280250,1.000000


In [65]:
df3_pre.head(1)

,충전소,충전일수,충전량(kWh),CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km,일수당충전량
0,G-Tower(경제자유구역청),408,30259.44,56,127,213,288,358,74.165294


In [66]:
df_q = pd.read_csv("버퍼2km_qgis_join.csv", encoding = "CP949")
df_q.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,경제인구,변압기,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,0.0,0.0,0.0,0.0,2761.0,24.0,27526.0,67.0


In [67]:
df_q = pd.merge(df_q, df3_pre.iloc[:,[0,1,2,8]], left_on = '충전소_명', right_on = '충전소')
df_q

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,경제인구,변압기,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt,충전소,충전일수,충전량(kWh),일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,0.0,0.0,0.0,0.0,2761.0,24.0,27526.0,67.0,인천서구청 제1주차장,206,3224.46,15.652718
1,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,연수구,126.66727,37.38151,121,0.0,0.0,0.0,0.0,2597.0,26.0,62531.0,38.0,연세대학교 국제캠퍼스,656,53172.87,81.056204
2,중부지방고용노동청 별관 앞,인천광역시 남동구 구월3동 1111-6 중부지방고용노동청 별관 앞,남동구,126.69954,37.45113,103,0.0,0.0,0.0,0.0,10903.0,65.0,17727.0,101.0,중부지방고용노동청 별관 앞,650,53635.05,82.515462
3,강화풍물시장 공영주차장,인천광역시 강화군 강화읍 갑곳리 849 고엽제전우회 건물 옆,강화군,126.49287,37.74142,21,0.0,0.0,0.0,0.0,146.0,2.0,659.0,5.0,강화풍물시장 공영주차장,597,27766.94,46.510787
4,영종대교 휴게소 주차장,인천광역시 서구 경서동 산 257-4 지상 버스주차장 옆,서구,126.60708,37.55390,1,0.0,0.0,0.0,0.0,463.0,3.0,NaN,NaN,영종대교 휴게소 주차장,578,30137.35,52.140744
5,인천공항 정부합동청사,"인천광역시 중구 공항로424번길 47 , 정문 주차장",중구,126.46134,37.44039,10,0.0,0.0,0.0,0.0,NaN,NaN,295.0,1.0,인천공항 정부합동청사,437,8148.95,18.647483
6,검단2공영주차장,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,서구,126.67502,37.59202,36,0.0,0.0,0.0,0.0,955.0,8.0,6599.0,15.0,검단2공영주차장,630,44358.60,70.410476
7,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",강화군,126.49913,37.74187,18,0.0,0.0,0.0,0.0,88.0,1.0,659.0,5.0,강화지사,342,10877.10,31.804386
8,계산1택지 공영주차장,인천광역시 계양구 계산동 1060 구두수선방 옆,계양구,126.73676,37.54099,114,0.0,0.0,0.0,0.0,5482.0,33.0,54660.0,77.0,계산1택지 공영주차장,606,27959.93,46.138498
9,계양구 공영주차장,"인천광역시 계양구 봉오대로 679-1 , 작전역 2번 출구 100m 앞",계양구,126.72446,37.53091,138,0.0,0.0,0.0,0.0,8195.0,54.0,54477.0,101.0,계양구 공영주차장,579,26312.12,45.444076


In [68]:
df_q2 = pd.read_csv("버퍼2km_qgis_join_수정.csv", encoding = "CP949")
df_q2

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,도서관,민행기,변압기
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,0.0,0.0,2761.0,24.0,27526.0,67.0,66686.0,9,20,331
1,인천 학익풍림아이원 아파트,인천광역시 미추홀구 매소홀로 340,미추홀구,126.66147,37.44042,61,0.0,0.0,10453.0,25.0,87364.0,105.0,144322.0,5,45,641
2,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,연수구,126.66727,37.38151,121,0.0,0.0,2597.0,26.0,62531.0,38.0,59740.0,2,20,317
3,중부지방고용노동청 별관 앞,인천광역시 남동구 구월3동 1111-6 중부지방고용노동청 별관 앞,남동구,126.69954,37.45113,103,0.0,0.0,10903.0,65.0,17727.0,101.0,232312.0,3,68,1383
4,강화풍물시장 공영주차장,인천광역시 강화군 강화읍 갑곳리 849 고엽제전우회 건물 옆,강화군,126.49287,37.74142,21,0.0,0.0,146.0,2.0,659.0,5.0,14641.0,1,17,145
5,영종대교 휴게소 주차장,인천광역시 서구 경서동 산 257-4 지상 버스주차장 옆,서구,126.60708,37.55390,1,0.0,0.0,463.0,3.0,NaN,NaN,1193.0,0,1,63
6,더샵퍼스트월드,인천광역시 연수구 해돋이로 107,연수구,126.64765,37.39279,186,0.0,0.0,6500.0,43.0,104796.0,75.0,86413.0,3,27,358
7,인천공항 정부합동청사,"인천광역시 중구 공항로424번길 47 , 정문 주차장",중구,126.46134,37.44039,10,0.0,0.0,NaN,NaN,295.0,1.0,488.0,0,0,30
8,검단2공영주차장,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,서구,126.67502,37.59202,36,0.0,0.0,955.0,8.0,6599.0,15.0,68579.0,9,17,421
9,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",강화군,126.49913,37.74187,18,0.0,0.0,88.0,1.0,659.0,5.0,13732.0,1,14,138


In [69]:
df_q3 = pd.merge(df_q2, df3_pre.iloc[:,[0,1,2,8]], left_on = '충전소_명', right_on = '충전소')
df_q3.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,도서관,민행기,변압기,충전소,충전일수,충전량(kWh),일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,0.0,0.0,2761.0,24.0,27526.0,67.0,66686.0,9,20,331,인천서구청 제1주차장,206,3224.46,15.652718


In [70]:
df_q3['관공서'] = df_q3['도서관'] + df_q3['민행기'] 

In [71]:
df_q3.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,도서관,민행기,변압기,충전소,충전일수,충전량(kWh),일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,29,0.0,2761.0,24.0,27526.0,67.0,66686.0,9,20,331,인천서구청 제1주차장,206,3224.46,15.652718


In [72]:
del df_q3['충전소']
del df_q3['일수']
del df_q3['도서관']
del df_q3['민행기']
df_q3.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,변압기,충전일수,충전량(kWh),일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,29,2761.0,24.0,27526.0,67.0,66686.0,331,206,3224.46,15.652718


In [73]:
len(df_q3)

85

In [74]:
df_q3[df_q3['충전소_명'] == '롯데마트 영종도점']['주소'] = ['인천광역시 중구 흰바위로 51 지상주차장 4층'] * 2

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [75]:
len(df_q3)

85

In [76]:
df_q3 = df_q3.drop_duplicates(['충전소_명'], keep = 'first')

In [77]:
len(df_q3)

84

In [78]:
df_q3.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,변압기,충전일수,충전량(kWh),일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,29,2761.0,24.0,27526.0,67.0,66686.0,331,206,3224.46,15.652718


In [79]:
df_q3.isnull().sum()

충전소_명         0
주소            0
시군구명          0
X좌표           0
Y좌표           0
car_num_2k    0
관공서           0
일반주차면         6
일반cnt         6
집합주차면         9
집합cnt         9
경제인구          0
변압기           0
충전일수          0
충전량(kWh)      0
일수당충전량        0
dtype: int64

In [80]:
df_q3 = df_q3.fillna(0)

# 2km 버퍼 내에 Columns들 수 간의 상관관계

In [81]:
df_q3.iloc[:, 5:].corr()

,car_num_2k,관공서,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,변압기,충전일수,충전량(kWh),일수당충전량
car_num_2k,1.000000,0.695650,0.676250,0.773880,0.445998,0.393145,0.708763,0.628749,0.227569,0.295619,0.298913
관공서,0.695650,1.000000,0.758889,0.781425,0.358222,0.445154,0.873761,0.854804,0.160095,0.154909,0.132668
일반주차면,0.676250,0.758889,1.000000,0.898393,0.316623,0.389647,0.645846,0.787720,0.181664,0.216877,0.179061
일반cnt,0.773880,0.781425,0.898393,1.000000,0.511417,0.575023,0.730473,0.880345,0.195303,0.279348,0.268755
집합주차면,0.445998,0.358222,0.316623,0.511417,1.000000,0.846176,0.490689,0.477901,0.041408,0.050815,0.065253
집합cnt,0.393145,0.445154,0.389647,0.575023,0.846176,1.000000,0.507516,0.578016,0.100962,0.125129,0.146126
경제인구,0.708763,0.873761,0.645846,0.730473,0.490689,0.507516,1.000000,0.843954,0.152046,0.157670,0.139532
변압기,0.628749,0.854804,0.787720,0.880345,0.477901,0.578016,0.843954,1.000000,0.144530,0.187978,0.161911
충전일수,0.227569,0.160095,0.181664,0.195303,0.041408,0.100962,0.152046,0.144530,1.000000,0.842862,0.534958
충전량(kWh),0.295619,0.154909,0.216877,0.279348,0.050815,0.125129,0.157670,0.187978,0.842862,1.000000,0.854195
